In [2]:
import neurokit2 as nk
import numpy as np
import pandas as pd

In [9]:
ids = ['P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10']

In [12]:
raw_emg = []
for subject_id in ids:
    raw_emg.append(pd.read_csv(f"data\\BSL-EMG-Book{subject_id}.csv", encoding="utf-8"))

In [14]:
df_raw = pd.DataFrame(columns=ids)

In [19]:
len(raw_emg)
for e in raw_emg:
    print(len(e))
 # if len e is less then max of raw_emg length, fill with zeros to the same size

40090
32941
34972
35490
35364
34918
38880
33866
33505


In [ ]:
def check_time(subject):
# measure length of mV data, divide by fs = 500 Hz (samples / samples/seconds = seconds)